In [7]:
import os
import pandas as pd 
import torch
from torch import nn
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
from pathlib import Path


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [11]:
class CustomDataset(Dataset):
    def __init__(self, csv_file,img_dir, transforms=None):
        self.annotations = pd.read_csv(self.csv_file)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.annotations.iloc[idx, 1], dtype=torch.long)

        if self.transforms:
            image = self.transforms(image)

        return image, label
        

In [4]:
detailed_class = pd.read_csv("stage_2_detailed_class_info.csv")
detailed_class

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [5]:
train_labels = pd.read_csv("stage_2_train_labels.csv")
train_labels

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [6]:
sample_submission = pd.read_csv("stage_2_sample_submission.csv")
sample_submission

,patientId,PredictionString
0,0000a175-0e68-4ca4-b1af-167204a7e0bc,0.5 0 0 100 100
1,0005d3cc-3c3f-40b9-93c3-46231c3eb813,0.5 0 0 100 100
2,000686d7-f4fc-448d-97a0-44fa9c5d3aa6,0.5 0 0 100 100
3,000e3a7d-c0ca-4349-bb26-5af2d8993c3d,0.5 0 0 100 100
4,00100a24-854d-423d-a092-edcf6179e061,0.5 0 0 100 100
...,...,...
2995,c1e88810-9e4e-4f39-9306-8d314bfc1ff1,0.5 0 0 100 100
2996,c1ec035b-377b-416c-a281-f868b7c9b6c3,0.5 0 0 100 100
2997,c1ef5b66-0fd7-49d1-ae6b-5af84929414b,0.5 0 0 100 100
2998,c1ef6724-f95f-40f1-b25b-de806d9bc39d,0.5 0 0 100 100


In [9]:
DATA_DIR = Path("stage_2_train_images/")

In [10]:
with open(os.path.join(DATA_DIR,'stage_2_train_labels_right.csv'), newline="") as inp, open(os.path.join(DATA_DIR,'stage_2_train_labels_right_poprawne.csv'), 'w', newline="") as out:
    reader = csv.DictReader(inp, delimiter=' ',fieldnames = ['patientId','x', 'y', 'width', 'height', 'Target'])
    writer = csv.DictWriter(out, fieldnames=reader.fieldnames)
    # read input file fully into a list of rows
    reader = list(reader)
    writer.writeheader()
    # now iterate on reversed list
    for row in reversed(reader):
   

SyntaxError: incomplete input (974121509.py, line 9)

In [ ]:
transforms = 